In [0]:
%%writefile run_bayenv.ipy

import psutil
import numpy as np
import sys
import os
import time

def get_free_cpu():
    procs = {}
    affinities = set()
    cpus = !cat /proc/cpuinfo | grep processor | wc -l
    print("cpus = %s" % cpus)
    try:
        cpus = int(cpus[0])
    except ValueError as e:
        cpus = int(cpus[2]) #account for stupid cluster error with BASH_FUNC_module
    possible_cpus = set(range(cpus))
    for proc in psutil.process_iter():
        try:
           if proc.username() == 'cfriedline' and proc.name() == 'bayenv2':
            procs[proc.pid] = proc.cpu_affinity()
            [affinities.add(x) for x in proc.cpu_affinity()]
        except psutil.NoSuchProcess:
            pass
    available_cpus = possible_cpus.difference(affinities)
    if len(available_cpus) == 0:
        print("no cpus available.  trying again...")
        time.sleep(1)
        return(get_free_cpu())
    return np.random.choice(list(available_cpus))

cmd = "numactl --physcpubind=%d %s" % (get_free_cpu(), sys.argv[1])
print(cmd)
!{cmd}